## 加载

* #### 导入库

In [1]:
import pandas as pd
import numpy as np
from pyecharts import Bar,Line,Grid,Radar
import pyecharts
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import cross_val_score,cross_val_predict
from sklearn.svm import LinearSVC,SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from pyecharts import Geo
from sklearn.preprocessing import LabelEncoder,Binarizer,OneHotEncoder

D:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


* #### 导入数据

In [2]:
%%time
data = pd.read_excel('大众点评一二线城市美食07-26.xlsx')

Wall time: 1min 9s


## 数据整理

* #### 查看数据

共有585915条数据，10个维度

In [3]:
data.head()

,City,Cuisine,Name,Star,Comments,PCC,Taste,Environment,Service,Addr
0,北京,私房菜,川E,45,339.0,93.0,8.6,9.1,8.7,东四五条139号
1,北京,私房菜,如壹私厨,45,152.0,68.0,8.4,9.0,8.7,798艺术区陶瓷二街
2,北京,私房菜,宴桃园餐厅(门头沟店),45,2391.0,70.0,8.5,9.1,8.8,冯村华润置地puls365二楼207室
3,北京,私房菜,海盗虾饭(慈云寺店)分店,45,376.0,45.0,8.4,8.7,8.6,朝阳路八里庄东里1号A区112A
4,北京,私房菜,三合楼(三源里街店),45,1508.0,62.0,8.1,8.2,7.7,东三环三元桥三源里街23号


In [4]:
print(data.shape)

(585915, 10)


* #### 数据去重

然而这些数据中有不少重复值，鉴于餐厅可能有分店所以名字重复可以理解（其实不能，分店也要加上XX分店呀~不过好像大众点评没那么严格？），所以我选取了地址作为衡量标准，无论如何地址都应该是唯一的，如果两家店地址重复了，那我只能认为他们是一家店了~ 这样经过去重后，发现数量由585915降到了516674

In [5]:
print('未去重地址数',len(data.Addr))
print('唯一地址数',len(data.Addr.unique()))

未去重地址数 585915
唯一地址数 516674


* #### 查看去重后数据

In [6]:
data_new = data.drop_duplicates('Addr')
data_new.head()

,City,Cuisine,Name,Star,Comments,PCC,Taste,Environment,Service,Addr
0,北京,私房菜,川E,45,339.0,93.0,8.6,9.1,8.7,东四五条139号
1,北京,私房菜,如壹私厨,45,152.0,68.0,8.4,9.0,8.7,798艺术区陶瓷二街
2,北京,私房菜,宴桃园餐厅(门头沟店),45,2391.0,70.0,8.5,9.1,8.8,冯村华润置地puls365二楼207室
3,北京,私房菜,海盗虾饭(慈云寺店)分店,45,376.0,45.0,8.4,8.7,8.6,朝阳路八里庄东里1号A区112A
4,北京,私房菜,三合楼(三源里街店),45,1508.0,62.0,8.1,8.2,7.7,东三环三元桥三源里街23号


* #### 处理缺失值

名字字段有11个缺失值，鉴于名字是一个餐厅必不可少的要素，所以对这11家缺少名字的餐厅删除（你们什么态度，连名字都懒得起？）地址同理，没地址我可找不到你啊。然后针对于口味、环境、服务以及评论数和人均消费都存在缺失值的数据，我本来是想打算给他们回填0作为数值的，但是转念一想，我是想找出优秀的餐厅啊，那么这些“三无”餐厅对于整体分析没有帮助，是冗余值啊。而且还会在计算平均值等各项指标时造成误差，所以我决定狠心把他们都删了吧。当然有些“三无”餐厅可能只是刚刚开业的新餐厅，确实会存在一部分的误杀情况。不过新餐厅还是有风险的，我这次只想来个保险点的分析。So，等你攒够了足够的评分再来入选吧，相信是金子总会发光的。

In [7]:
data_new.apply(lambda x:x.isnull().sum())

City                0
Cuisine             0
Name               11
Star                0
Comments       123723
PCC            255450
Taste          130443
Environment    130437
Service        130438
Addr                1
dtype: int64

删去缺失值后，数据减少了差不多一半，这说明至少在这次的数据集中大众点评上将近一半的店铺是有缺失关键信息的，不能对我们以很好的指引。点评君你还要加油啊~

In [8]:
data_clear = data_new.dropna()
print('去除缺失值后还剩：%s间饭店'%len(data_clear))

去除缺失值后还剩：247431间饭店


In [9]:
data_clear.apply(lambda x:x.isnull().sum())

City           0
Cuisine        0
Name           0
Star           0
Comments       0
PCC            0
Taste          0
Environment    0
Service        0
Addr           0
dtype: int64

* #### 构建新特征

为了后面在分析口味等评分时方便，我在这里构建了一个新的特征——综合分，既‘口味’、‘环境’及‘服务’三者的平均分，精度取一位小数

In [10]:
data_clear = data_clear.copy()
data_clear['overall'] = round((data_clear['Service']+data_clear['Environment']+data_clear['Taste'])/3,1)

* #### 查看各类目都包含什么

In [11]:
for i in data_clear:
    print("\n%s 类别包含以下信息~~"%i)
    print(data_clear[i].value_counts())


City 类别包含以下信息~~
北京      12522
上海      10892
广州       9454
深圳       9425
天津       7944
苏州       7587
成都       7388
南京       7319
杭州       7010
武汉       6668
西安       6450
郑州       6174
青岛       6050
重庆       6046
沈阳       5866
大连       5337
无锡       5196
厦门       5164
济南       5120
宁波       5100
佛山       5061
哈尔滨      4793
东莞       4755
长沙       4644
合肥       4638
福州       4430
长春       4427
常州       4114
石家庄      4054
温州       3918
烟台       3869
泉州       3797
南通       3606
昆明       3590
金华       3300
乌鲁木齐     3224
惠州       3219
中山       3196
嘉兴       3184
南昌       3152
太原       3148
南宁       3013
徐州       2987
珠海       2942
绍兴       2927
贵阳       2904
台州       2881
海口       2526
兰州       2420
Name: City, dtype: int64

Cuisine 类别包含以下信息~~
面包甜点      22684
火锅        22030
西餐        18110
川菜        17591
其他        14618
小吃快餐      14435
烧烤        12832
咖啡厅       12507
海鲜         9278
粤菜         8230
日本料理       8188
韩国料理       7014
湘菜         6619
面馆         6601
快餐简餐       6377
东北菜        6

## 数据分析（EDA）

* ### 全国

可以看出餐厅数量最多的还是北上广深四个一线城市，其中帝都的数量是最多的共有12522间餐厅（帝都人民好幸福~）,最少的是兰州（呃，宇宙第一店兰州拉面的大本营啊，没想到数量最少）。广深两座城市数量基本持平，不愧是我专吃福建人的大吃省。这在一定程度上也反应出了城市的发展程度，毕竟民以食为天，人口净流入大的城市其饭店等基础民生的设施也更多。除北上广深四大一线城市外，像南京、天津、成都、杭州等明星二线城市饭店数量也很突出，都超过了平均值，而像一些欠发达地区饭店数量则较少，基本符合前面的分析~ 其实不难理解，像广阔的大西北除了旅游景点外都见不到几个人还怎么开店 哈哈（为啥我知道，，因为我刚刚去完回来啊，大西北的景色真心漂亮，强烈安利~）

In [12]:
index = [i for i in data_clear.groupby('City').count().index]
value = [int(i) for i in data_clear.groupby('City').count().Name.values]

bar = Bar("城市比较",subtitle="--餐厅数量",subtitle_color='black',title_pos='center',width=1000)
bar.add('',index,value,mark_line=['min','max','average'],xaxis_interval=0,xaxis_rotate=90,label_color=['orangered'],
        mark_point=['min'],mark_point_textcolor='blue')

bar

再来看看哪个城市菜品对多样化，菜系最多。一线城市依然名列前茅，符合预期。帝都力压群雄，拔得头筹，不过令我感到惊讶的是天津竟然仅以一票之差区居第二。在我想来菜品最多样化的应该都是外来人口流入巨大的城市，因为城市聚集了天南海北各地的人，大家口味不同自然就有了不同菜系的市场。而在一个历史相对悠久，人口流动并不是特别大的城市，绵长的历史早就使得当地人培养出了自己的饮食文化，因而种类不会特别多，所以对天津这么高的成绩我表示出了惊讶。不过这也说明，天津作为直辖市之一，发展的越来越好了，潜力巨大。与此相似的还有成都、南京、杭州等。同时，菜系种类的多寡也能反映出一个城市的包容度及本土文化的强势程度。种类多说明城市的包容性强，而种类少说明本土的饮食文化非常强势，挤压甚至同化了不少其他种类，譬如多少爽朗的北方爷们爱上了精致的广东早茶，又有多少吃不得辣的人一把鼻涕一把泪得在川菜馆里大快朵颐~（不过这一点在本次分析中存疑，因为按照这个思路下去的话北京的本土文化不够强势？长沙包容性最低？我以为并不见得~不过暂时我还没想明白是什么原因造成的这些认知偏差，留待后续改进思考，但我怀疑其中一个原因可能是因为本次数据并非全量数据造成的）

In [13]:
index = [i for i in data_clear.groupby('City').count().index]
value = [len(i) for i in data_clear.groupby('City')['Cuisine'].unique()]

bar = Bar("城市比较","--菜系种类",subtitle_color='blackt',title_pos='center',width=1000)
bar.add('',index,value,mark_line=['min','max','average'],xaxis_interval=0,xaxis_rotate=90,label_color=['orangered'],
        mark_point=['min'],mark_point_textcolor='blue')
bar

本次采样的49个城市中,可以发现饭店大多集中在东南沿海，尤其是长三角和珠三角地区，另外京津冀地区虽然并非沿海区域，但靠着帝都威名也同样聚集了大量的饭店。这也大体反应出了我国地域经济形势及发展态势，东南沿海发达，西北地区欠发达的形势并没有得到太大的改善，祖国的发展还需要靠我们来建设啊。另外，再次强调，本次分析数据并非全量数据，所以可能只是反映了一部分情况，真实情况或许有不同~

In [14]:

attr=[(i) for i in data_clear['City'].unique()]
value=[int(i) for i in data_clear.groupby('City').count().Name]

geo = Geo("全国主要城市餐厅数量", title_color="#fff", title_pos="center",
width=1000, height=600, background_color='#404a59')

geo.add("", attr, value, visual_range=[0, 10000], visual_text_color="right",
        is_visualmap=True,visual_type='size',visual_range_size=[10,30])

geo.add("", attr, value, type="heatmap", is_visualmap=True, visual_range=[0, 10000],
        visual_text_color='#fff')
geo

* ### 深圳

In [15]:
data_sz = data_clear[data_clear['City']=='深圳']
data_sz.head()

,City,Cuisine,Name,Star,Comments,PCC,Taste,Environment,Service,Addr,overall
366280,深圳,私房菜,三清一味健康私厨(万科星火店),45,160.0,109.0,8.2,9.0,8.4,五和大道南2号万科星火Online11栋,8.5
366281,深圳,私房菜,福福囍囍创意中国菜,40,57.0,99.0,7.7,8.5,8.4,购物公园北园A区一楼扶手梯旁八合里牛肉火锅楼下,8.2
366282,深圳,私房菜,十田味舍,45,49.0,88.0,8.4,8.6,8.6,大鹏古城南门刘屋巷12号,8.5
366283,深圳,私房菜,经典中央大街私房菜,40,1044.0,105.0,7.8,8.4,7.8,汕头街东部市场201号,8.0
366284,深圳,私房菜,下沙老兵牛蹄店,40,38.0,31.0,8.1,7.1,7.7,沙头街道下沙一坊69-101,7.6


星级分布符合正态分布，主要集中在3.5星附近，2星和5星的都很少（1星数据清理时已去除）。3.5星的基本上是其他星级之和，而从3.5星到4星的数量陡然下降，说明4星是一个非常大的瓶颈，想要突破对于大部分店家而言难度不小。绝大部分店家在达到一个平均水准（3.5星）后就停留在这里了，很难做的更好。而从3星到3.5星的陡然提升也说明3星是一个比较容易达到的星级。所以那些2星3星的店家~好好反省下吧 哈哈哈

In [16]:
index = [str(i) for i in data_sz.groupby('Star').count().index]
value = [int(i) for i in data_sz.groupby('Star').count().Name.values]

bar = Bar('星级分布',title_pos='center')
bar.add('',index,value,label_color=['yellowgreen'],is_label_show=True)

bar

说完了星级来看看人均。深圳饭店人均消费最高1806元，最低4元，平均67元。嗯，平均67元，好像跟我的感觉差不多，基本上每次跟小伙伴们出去吃饭人均都在七八十到一百多吧，比平均值略高一点。唉，收入没有跑赢平均值，吃饭消费倒是跑赢了。。。

再来看看分布情况，非常明显的一个右偏分布。。绝大部分人人均消费都是少于200元，普遍集中在100元以内（看到这里我心里又平衡了）。然而还是有少部分人人均消费大于1000的，对于这部分人我只想说，土豪求抱大腿啊啊啊。。。 看来这又给我一个小目标了啊，先吃它1000块。这反应出了深圳这座城市中，大部分人都是普通的工薪阶级，人均消费能力低，但同时依然有少量的资产阶级拥有远超大众的消费能力，贫富差距很大，两极分化严重，这也完全符合二八原则。当然，深圳是一座寻梦的城市，虽然贫富差距大，但总有不服输的追梦者创造出了不少神话，在这个越来越定型的社会中脱离了自身所处阶级，实现了阶级跃迁~

当然，出于好奇，我看了下最贵的是哪家餐厅。额，西安老刘家？1806元？我怎么没听过这家店。。。于是我去大众点评看了下。。好吧画风是这个样子的。。
这充分告诉我们，有时候数据是不可信的，这数据太离谱了，不知道大众点评是怎么得出这样的数据的

### 人均

In [17]:
print('最贵',data_sz['PCC'].max())
print('最低',data_sz['PCC'].min())
print('平均',data_sz['PCC'].mean())

最贵 1806.0
最低 4.0
平均 66.94928381962865


In [18]:
PCC = data_sz['PCC']
bins = [0,50,100,200,300,400,500,600,700,800,900,1000,2000]
group_name = ['小于50','51-100','101-200','201-300','301-400','401-500','501-600','601-700','701-800','801-900','901-1000','大于1000']

cuts = pd.cut(PCC,bins,labels=group_name)
index = group_name
value = [int(i) for i in pd.value_counts(cuts)]

bar = Bar('人均消费分布',title_pos='center')
bar.add('',index,value,label_color=['mediumpurple'],is_label_show=True,xaxis_interval=0,xaxis_rotate=30)

bar

In [19]:
print('价格最贵的是：\n\n',data_sz.loc[data_sz['PCC'].argmax()])

价格最贵的是：

 City                      深圳
Cuisine                  西北菜
Name                   西安老刘家
Star                      35
Comments                  31
PCC                     1806
Taste                    7.1
Environment              6.8
Service                  6.9
Addr           前海路诺德国际花园110铺
overall                  6.9
Name: 375122, dtype: object


接下来我们看看人气情况。在这里，我把评论数的多少作为判定其人气高低的指标，毕竟人气高的话点评的人也会多，而无人问津的店自然也就没什么人点评~虽然这样判断不是很准确，但至少应该是正相关的，所以在没有更好的办法前就勉强拿来一用吧~(当然，如果一家店特别坑，也会导致大量的评论前来吐槽，但负面人气也是人气啊~)

那么评论最多的店有20094条评论，而最少的只有1条评论，平均是294条。从图一可以看到只有一家店评论超过了2万，基本上大家还都是在1万条以下的。针对2万条评论的这家店我看了下，是“ 幸福西饼生日蛋糕(布心店)分店”。这么高的评论数不像是自然情况下的产生的，应该是有人为干扰因素，怀疑是店家搞的什么活动，类似于评论返现啊之类的造成的。

通常情况下，评论越多也就意味着店面规模或者品牌越大型，因为只有大型的店面，才能有更高的知名度，吸引更多的人过来就餐。小店面在这一点上是无法与大店媲美的，当然也会有那种几十年的老店就是一个小小的店面，然后人们口口相传名声打开了，但这种情况太少了，个例我们就不予考虑了。那么我们看一下分布情况，跟人均消费一样，依然是个右偏分布，绝大部分都集中在100条以下。也就是绝大部分店面都是小店，真正能做大的、做成连锁品牌的店面非常的少，看来成功总是属于少部分人啊~

### 最火爆，人气最高（评论数最多）

In [20]:
print('最多',data_sz['Comments'].max())
print('最少',data_sz['Comments'].min())
print('平均',data_sz['Comments'].mean())

最多 20094.0
最少 1.0
平均 293.782599469496


In [21]:
# Line
index = [i for i in data_sz['Name']]
value = [int(i) for i in data_sz['Comments'].values]

line = Line('评论数量')
line.add('',index,value,mark_line=['max','min','average'],is_smooth=True)

# 离散化和面元划分
comments = data_sz['Comments']
bins = [0,100,200,300,500,1000,2000,5000,10000,30000]
group_name = ['<100','101-200','201-300','301-500','501-1000','1001-2000','2001-5000','5001-10000','>10000']

cuts = pd.cut(comments,bins,labels=group_name)

# Bar
index = group_name
value = [int(i) for i in pd.value_counts(cuts)]

bar = Bar('评论分布',title_top='middle',height=600)
bar.add('',index,value,xaxis_interval=0,xaxis_rotate=30,is_label_show=True,mark_line=['max'],
        label_color=['deepskyblue','#698088'])

# 并行显示多图
grid = Grid()
grid.add(bar,grid_top='60%')
grid.add(line,grid_bottom='60%')
grid

In [22]:
print('人气最高的是：\n\n',data_sz.loc[data_sz['Comments'].argmax()])

人气最高的是：

 City                            深圳
Cuisine                       面包甜点
Name               幸福西饼生日蛋糕(布心店)分店
Star                            45
Comments                     20094
PCC                             67
Taste                          8.5
Environment                    8.6
Service                        8.7
Addr           金稻田路边防布心住宅区A座3号4号商铺
overall                        8.6
Name: 382086, dtype: object


把口味、环境、服务及之前构建的综合指标放在一起查看其分布情况，可看出每一个指标都是呈正态分布的，大部分集中在7.2到7.6之间，剩下两端的分布很少。然后我发现了一个有意思的现象，在2星的时候，环境比其他项的评分均分要高，这说明在星级较低的餐厅就餐时人们对于其环境的容忍度是比较高的。这或许是因为大家去一个低星级餐厅时已经对其环境有一定的心理预设。而随着星级越来越高，各项指标也越发平衡，说明饭店如果想得到更高的星级必须要均衡发展，不能偏科。同时，我还注意到5星级的饭店其服务得分会略微高于其他项，这说明越高星级的餐厅越重视服务水平。当然换句话说，这能否说明我们现在的很多餐厅，尤其是高档餐厅，相比于食物本身的味道它们更注重形式和服务表现。

### 各项得分

In [23]:
index = [str(i) for i in data_sz.groupby('Star').count().index]

value1 = [round(float(data_sz[data_sz['Star']==i].groupby('Star')['Taste'].mean().values),2) for i in data_sz.groupby('Star').count().index]
value2 = [round(float(data_sz[data_sz['Star']==i].groupby('Star')['Environment'].mean().values),2) for i in data_sz.groupby('Star').count().index]
value3 = [round(float(data_sz[data_sz['Star']==i].groupby('Star')['Service'].mean().values),2) for i in data_sz.groupby('Star').count().index]
value4 = [round(float(data_sz[data_sz['Star']==i].groupby('Star')['overall'].mean().values),2) for i in data_sz.groupby('Star').count().index]

bar1 = Bar('各项得分',subtitle='分数',height=600,width=1000,title_top='middle')
bar1.add('口味',index,value1,xaxis_interval=0,mark_line=['max'],
        is_random=True)
bar1.add('环境',index,value2,xaxis_interval=0,xaxis_rotate=20,mark_line=['max'],
        is_random=True,legend_pos='right')
bar1.add('服务',index,value3,xaxis_interval=0,xaxis_rotate=20,mark_line=['max'],
        is_random=True)
bar1.add('综合',index,value4,xaxis_rotate=20,mark_line=['max'],
        is_random=True,legend_pos='right',is_datazoom_show=False,is_more_utils=True)

# Bar
index1 = [str(i) for i in data_sz.groupby('Taste').count().index]
value1 = [int(i) for i in data_sz.groupby('Taste').count().Name.values]

index2 = [str(i) for i in data_sz.groupby('Environment').count().index]
value2 = [int(i) for i in data_sz.groupby('Environment').count().Name.values]

index3 = [str(i) for i in data_sz.groupby('Service').count().index]
value3 = [int(i) for i in data_sz.groupby('Service').count().Name.values]

index4 = [str(i) for i in data_sz.groupby('overall').count().index]
value4 = [int(i) for i in data_sz.groupby('overall').count().Name.values]

bar2 = Bar('各项得分',subtitle='星级',height=1000,width=1000)
bar2.add('口味',index1,value1,xaxis_interval=0,mark_line=['max'],
        is_random=True)
bar2.add('环境',index2,value2,xaxis_interval=0,xaxis_rotate=20,mark_line=['max'],
        is_random=True,legend_pos='right')
bar2.add('服务',index3,value3,xaxis_interval=0,xaxis_rotate=20,mark_line=['max'],
        is_random=True)
bar2.add('综合',index4,value4,xaxis_rotate=20,mark_line=['max'],
        is_random=True,legend_pos='right',is_datazoom_show=False,is_more_utils=True)

grid = Grid()
grid.add(bar2,grid_top='55%')
grid.add(bar1,grid_bottom='60%')
grid

看起来深圳的各项指标还是挺均衡的

In [24]:
schema = [ 
    ("星级", int(data_sz['Star'].max())),  ("综合", 10),
    ("口味", 10), ("环境",10), ("服务", 10)
]
v1 = [[int(data_sz['Star'].mean()),  round(float(data_sz['overall'].mean()),2), 
       round(float(data_sz['Taste'].mean()),2), round(float(data_sz['Environment'].mean()),2), round(float(data_sz['Service'].mean()),2)]]

radar = Radar(title='深圳平均水准')
radar.config(schema)
radar.add("", v1, is_splitline=False, is_axisline_show=False,is_random=True,is_label_show=True)

radar

好的，我知道大家一定很想知道这几项指标最高的分别都是那些店面~~

In [25]:
print('综合得分最高的是：\n\n',data_sz.loc[data_sz['overall'].argmax()])
print('\n口味得分最高的是：\n\n',data_sz.loc[data_sz['Taste'].argmax()])
print('\n环境得分最高的是：\n\n',data_sz.loc[data_sz['Environment'].argmax()])
print('\n服务得分最高的是：\n\n',data_sz.loc[data_sz['Service'].argmax()])

综合得分最高的是：

 City                         深圳
Cuisine                     小龙虾
Name                    天宝兄弟虾蟹蛇
Star                         50
Comments                    129
PCC                         161
Taste                       9.2
Environment                 9.1
Service                     9.1
Addr           上梅林梅华路梅华1号楼首层最东侧
overall                     9.1
Name: 367882, dtype: object

口味得分最高的是：

 City                         深圳
Cuisine                     小龙虾
Name                    天宝兄弟虾蟹蛇
Star                         50
Comments                    129
PCC                         161
Taste                       9.2
Environment                 9.1
Service                     9.1
Addr           上梅林梅华路梅华1号楼首层最东侧
overall                     9.1
Name: 367882, dtype: object

环境得分最高的是：

 City                                深圳
Cuisine                             粤菜
Name                                T馆
Star                                40
Comments                           727
PCC    

那么什么菜系最贵呢~来看看吧 海鲜 有木有想到呢
那最受欢迎的菜系咧~ 竟然是江浙菜 讲道理，我还以为是早茶呢~毕竟我大广东不是都喝早茶的么

### 什么菜系最贵，最受欢迎

In [26]:
# 最贵
print('深圳平均最贵的菜系是：%s'%data_sz.groupby('Cuisine').mean()['PCC'].argmax(),',平均人均要%.2f元'%data_sz.groupby('Cuisine').mean()['PCC'].max())

# 最受欢迎
print('深圳平均最受欢迎的菜系是：%s'%data_sz.groupby('Cuisine').mean()['Comments'].argmax(),',平均有%d条评论'%data_sz.groupby('Cuisine').mean()['Comments'].max())


深圳平均最贵的菜系是：海鲜 ,平均人均要106.82元
深圳平均最受欢迎的菜系是：江浙菜 ,平均有902条评论


查看星级与评分之间的关系，发现呈正相关，星级越高综合评分越高

In [27]:
from pyecharts import Scatter

v1 = [float(i) for i in data_sz['Star']]
v2 = [float(i) for i in data_sz['overall']]
es = Scatter("星级-评分")
es.add("评分", v1, v2,is_more_utils=True)
es

In [28]:
from pyecharts import Boxplot

boxplot = Boxplot("箱形图")
x_axis = [str(i) for i in data_sz.groupby('Star').count().index]
y_axis1 = [data_sz[data_sz['Star']==i]['PCC'] for i in data_sz['Star'].unique()]


boxplot.add("category1", x_axis, boxplot.prepare_data(y_axis1))


boxplot

# 机器学习

想通过各项评分来预测星级，这是一个多分类问题，由于时间和精力有限，我决定把它转化为一个二分类问题，既判断一个餐厅是否为好餐厅。我将'Star'二值化，阈值取39，既简单的取星级4星以上的为好餐厅，以下的为不好的。然后将菜系由非数值标签处理为数值标签，本来应该再进行哑变量处理的，因为转为数值标签后，会有潜在顺序关系。但是不知道为何，进行了哑变量处理后，后面筛选特征重要性的时候就出问题了，由于时间紧张，我后来就放弃了哑变量处理。所以这里多少是会有点偏差的。

In [29]:
%%time
# data_sz['Great']=0
# data_sz.loc[data_sz['Star']>=45,'Great']=1
binarizer = Binarizer(threshold=39)
data_sz[['Star']] = binarizer.transform(data_sz[['Star']])
le = LabelEncoder()
data_sz['Cuisine'] = le.fit_transform(data_sz[['Cuisine']])
# enc = OneHotEncoder()
# data_sz['Cuisine'] = enc.fit_transform(a.reshape(-1,1))

Wall time: 832 ms


D:\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
D:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWi

In [30]:
data_sz.head()

,City,Cuisine,Name,Star,Comments,PCC,Taste,Environment,Service,Addr,overall
366280,深圳,18,三清一味健康私厨(万科星火店),1,160.0,109.0,8.2,9.0,8.4,五和大道南2号万科星火Online11栋,8.5
366281,深圳,18,福福囍囍创意中国菜,1,57.0,99.0,7.7,8.5,8.4,购物公园北园A区一楼扶手梯旁八合里牛肉火锅楼下,8.2
366282,深圳,18,十田味舍,1,49.0,88.0,8.4,8.6,8.6,大鹏古城南门刘屋巷12号,8.5
366283,深圳,18,经典中央大街私房菜,1,1044.0,105.0,7.8,8.4,7.8,汕头街东部市场201号,8.0
366284,深圳,18,下沙老兵牛蹄店,1,38.0,31.0,8.1,7.1,7.7,沙头街道下沙一坊69-101,7.6


## 划分feature和label

划分特征和标签。Star为标签，然后选'Cuisine','Comments','PCC','Taste','Environment','Service'作为特征。为什么不选City呢，因为城市都是深圳没有意义。选Name的原因很简单，这不是算卦，咱不看面相。不选Addr也是一个道理，咱也不看风水。overall是因为该分值就是由'Taste','Environment','Service'三个指标的均值构成的，相关性很高，可以舍去

然后从这几个特征中在筛选特征，找出特征重要性最高的80%的特征，为'Comments','Taste','Environment','Service'这四个特征

In [31]:
data_features = data_sz[['Cuisine','Comments','PCC','Taste','Environment','Service']].values
data_labels = data_sz['Star'].values

In [32]:
from sklearn.feature_selection import SelectKBest,SelectPercentile

selection = SelectPercentile(percentile=80)
selection.fit(data_features,data_labels)
selection.transform(data_features)
print(selection.get_support())
print(selection.scores_)
features_new = data_features[:,selection.get_support()]
# test_data_new = test_data.loc[:,selection.get_support()]
features_new

[False  True False  True  True  True]
[   131.22933967   1184.13265952    270.58460705  12124.36999202
   7170.69530397   9201.86654902]


array([[ 160. ,    8.2,    9. ,    8.4],
       [  57. ,    7.7,    8.5,    8.4],
       [  49. ,    8.4,    8.6,    8.6],
       ..., 
       [  41. ,    7.6,    7.5,    7.6],
       [ 179. ,    7.6,    7.6,    7.7],
       [  37. ,    7.8,    8.1,    7.6]])

In [33]:
# from sklearn.preprocessing import Binarizer
# Binarizer(threshold=3).fit_transform(data2)

## 特征缩放

因为评论数量和评分之间相差过大，所以进行特征缩放，防止某一过大值挤压其他过小的值。

In [34]:
from sklearn.preprocessing import MinMaxScaler

Scaler = MinMaxScaler()
data_features = Scaler.fit_transform(features_new)

### 交叉验证

In [35]:
# from sklearn.model_selection import KFold

# clf = LinearSVC()
# a = []
# kf = KFold(n_splits=10)
# for train_index,test_index in kf.split(data_features):
#     features_train, features_test = data_features[train_index], data_features[test_index]
#     labels_train, labels_test = data_labels[train_index], data_labels[test_index]
    
#     clf = clf.fit(features_train,labels_train)
#     score = clf.score(features_test,labels_test)
#     a.append(score)
#     print(score)
# sum(a)/len(a)

用交叉验证测量模型得分，cv取10

尝试了 LinearSVC、SVM、朴素贝叶斯、随机森林及XGBoost模型，测量下来XGBoost模型表现最好

### LinearSVC

In [36]:

clf = LinearSVC()
cross_val_score(clf,data_features,data_labels,cv=10).mean()

0.95023379859770252

### SVM

In [37]:
%%time

clf = SVC()
print(cross_val_score(clf,data_features,data_labels,cv=10).mean())

0.947265227675
Wall time: 4.51 s


### 朴素贝叶斯

In [38]:

clf = GaussianNB()
print(cross_val_score(clf,data_features,data_labels,cv=10).mean())

0.92816572435


### 随机森林

In [39]:

clf = RandomForestClassifier()
print(cross_val_score(clf,data_features,data_labels,cv=10).mean())

0.94131749719


### XGBoost

In [40]:
xgb = XGBClassifier()
print(cross_val_score(xgb,data_features,data_labels,cv=10).mean())

0.948853080453


In [41]:
%%time
param_test1 = {
    'max_depth':range(1,3),
    'min_child_weight':[i/10.0 for i in range(0,10)]
}
gs1 = GridSearchCV(estimator=XGBClassifier(learning_rate=0.1,max_depth=5,
                                min_child_weight=1,gamma=0,subsample=0.8,colsample_bytree=0.8,
                                objective='binary:logistic',nthread=4,scale_pos_weight=1,seed=27),
                param_grid=param_test1,scoring='roc_auc',n_jobs=4,iid=False,cv=10)
gs1.fit(data_features,data_labels)

Wall time: 16.6 s


In [42]:
gs1.grid_scores_,gs1.best_params_,gs1.best_score_

D:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.98878, std: 0.00687, params: {'max_depth': 1, 'min_child_weight': 0.0},
  mean: 0.98878, std: 0.00687, params: {'max_depth': 1, 'min_child_weight': 0.1},
  mean: 0.98878, std: 0.00687, params: {'max_depth': 1, 'min_child_weight': 0.2},
  mean: 0.98878, std: 0.00687, params: {'max_depth': 1, 'min_child_weight': 0.3},
  mean: 0.98878, std: 0.00687, params: {'max_depth': 1, 'min_child_weight': 0.4},
  mean: 0.98878, std: 0.00687, params: {'max_depth': 1, 'min_child_weight': 0.5},
  mean: 0.98878, std: 0.00687, params: {'max_depth': 1, 'min_child_weight': 0.6},
  mean: 0.98878, std: 0.00687, params: {'max_depth': 1, 'min_child_weight': 0.7},
  mean: 0.98878, std: 0.00687, params: {'max_depth': 1, 'min_child_weight': 0.8},
  mean: 0.98878, std: 0.00687, params: {'max_depth': 1, 'min_child_weight': 0.9},
  mean: 0.98925, std: 0.00672, params: {'max_depth': 2, 'min_child_weight': 0.0},
  mean: 0.98925, std: 0.00673, params: {'max_depth': 2, 'min_child_weight': 0.1},
  mean: 0.98921,

In [43]:
%%time
param_test2 = {
    'gamma':[i/100.0 for i in range(0,10)]
}
gs2 = GridSearchCV(estimator=XGBClassifier(learning_rate=0.1,max_depth=2,
                                min_child_weight=0,gamma=0,subsample=0.8,colsample_bytree=0.8,
                                objective='binary:logistic',nthread=4,scale_pos_weight=1,seed=27),
                param_grid=param_test2,scoring='roc_auc',n_jobs=4,iid=False,cv=10)
gs2.fit(data_features,data_labels)

Wall time: 11 s


In [44]:
gs2.grid_scores_,gs2.best_params_,gs2.best_score_

D:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.98925, std: 0.00672, params: {'gamma': 0.0},
  mean: 0.98925, std: 0.00672, params: {'gamma': 0.01},
  mean: 0.98926, std: 0.00673, params: {'gamma': 0.02},
  mean: 0.98926, std: 0.00673, params: {'gamma': 0.03},
  mean: 0.98926, std: 0.00673, params: {'gamma': 0.04},
  mean: 0.98926, std: 0.00673, params: {'gamma': 0.05},
  mean: 0.98926, std: 0.00673, params: {'gamma': 0.06},
  mean: 0.98926, std: 0.00673, params: {'gamma': 0.07},
  mean: 0.98926, std: 0.00673, params: {'gamma': 0.08},
  mean: 0.98926, std: 0.00673, params: {'gamma': 0.09}],
 {'gamma': 0.02},
 0.98925723305543622)

In [45]:
%%time
param_test3 = {
    'colsample_bytree':[i/10.0 for i in range(4,10)],
    'subsample':[i/10.0 for i in range(3,8)]
}
gs3 = GridSearchCV(estimator=XGBClassifier(learning_rate=0.1,max_depth=2,
                                min_child_weight=0,gamma=0.02,subsample=0.8,colsample_bytree=0.8,
                                objective='binary:logistic',nthread=4,scale_pos_weight=1,seed=27),
                param_grid=param_test3,scoring='roc_auc',n_jobs=4,iid=False,cv=10)
gs3.fit(data_features,data_labels)

Wall time: 22.5 s


In [46]:
gs3.grid_scores_,gs3.best_params_,gs3.best_score_

D:\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: 0.98865, std: 0.00722, params: {'colsample_bytree': 0.4, 'subsample': 0.3},
  mean: 0.98871, std: 0.00716, params: {'colsample_bytree': 0.4, 'subsample': 0.4},
  mean: 0.98874, std: 0.00711, params: {'colsample_bytree': 0.4, 'subsample': 0.5},
  mean: 0.98879, std: 0.00700, params: {'colsample_bytree': 0.4, 'subsample': 0.6},
  mean: 0.98880, std: 0.00707, params: {'colsample_bytree': 0.4, 'subsample': 0.7},
  mean: 0.98934, std: 0.00663, params: {'colsample_bytree': 0.5, 'subsample': 0.3},
  mean: 0.98934, std: 0.00659, params: {'colsample_bytree': 0.5, 'subsample': 0.4},
  mean: 0.98939, std: 0.00652, params: {'colsample_bytree': 0.5, 'subsample': 0.5},
  mean: 0.98935, std: 0.00663, params: {'colsample_bytree': 0.5, 'subsample': 0.6},
  mean: 0.98931, std: 0.00664, params: {'colsample_bytree': 0.5, 'subsample': 0.7},
  mean: 0.98934, std: 0.00663, params: {'colsample_bytree': 0.6, 'subsample': 0.3},
  mean: 0.98934, std: 0.00659, params: {'colsample_bytree': 0.6, 'subsample'

调参之前为0.9489，调参完后为0.9497，得分上升

In [47]:
xgb = XGBClassifier(learning_rate=0.1,
                    max_depth=2,
                    min_child_weight=0,
                    gamma=0.02,
                    subsample=0.5,
                    colsample_bytree=0.5,
                    objective='binary:logistic',
                    nthread=4,
                    scale_pos_weight=1,
                    seed=27)


In [48]:
cross_val_score(xgb,data_features,data_labels,cv=10).mean()

0.94970211220524337

最后预测一下~

In [49]:
xgb.fit(data_features,data_labels)
xgb.predict([100,9.2,9.3,8.2])

array([1], dtype=int64)